### 6. 综合案例

#### 1）垃圾邮件分类

- 数据集介绍：包含5000份正常邮件、5001份垃圾邮件的样本
- 文本特征处理方式：采用TF-IDF作为文本特征值
- 模型选择：朴素贝叶斯、支持向量机模型
- 基本流程：读取数据 → 去除停用词和特殊符号 → 计算TF-IDF特征值 → 模型训练 → 预测 → 打印结果

In [1]:
# -*- coding: utf-8 -*-
# 利用TF-IDF特征、朴素贝叶斯/支持向量机实现垃圾邮件分类
import numpy as np
import re
import string
import sklearn.model_selection as ms
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn import metrics

import jieba
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
label_name_map = ["垃圾邮件", "正常邮件"]


# 分词
def tokenize_text(text):
    tokens = jieba.cut(text)  # 分词
    tokens = [token.strip() for token in tokens]  # 去空格
    return tokens


def remove_special_characters(text):
    tokens = tokenize_text(text)
    # escape函数对字符进行转义处理
    # compile函数用于编译正则表达式，生成一个 Pattern 对象
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    # filter() 函数用于过滤序列，过滤掉不符合条件的元素，返回由符合条件元素组成的新列表
    # sub函数进行正则匹配字符串替换
    filtered_tokens = filter(None, [pattern.sub('', token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text


# 去除停用词
def remove_stopwords(text):
    tokens = tokenize_text(text)  # 分词、去空格
    filtered_tokens = [token for token in tokens if token not in stopword_list]  # 去除停用词
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text


# 规范化处理
def normalize_corpus(corpus):
    result = []  # 处理结果

    for text in corpus:  # 遍历每个词汇
        text = remove_special_characters(text)  # 去除标点符号
        text = remove_stopwords(text)  # 去除停用词
        result.append(text)

    return result


def tfidf_extractor(corpus):
    vectorizer = TfidfVectorizer(min_df=1,
                                 norm='l2',
                                 smooth_idf=True,
                                 use_idf=True)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features


def get_data():
    '''
    获取数据
    :return: 文本数据，对应的labels
    '''
    corpus = []  # 邮件内容
    labels = []  # 标签(0-垃圾邮件 1-正常邮件)

    # 正常邮件
    with open("ham_data.txt", encoding="utf-8") as f:
        for line in f.readlines():
            corpus.append(line)
            labels.append(1)

    # 垃圾邮件
    with open("spam_data.txt", encoding="utf-8") as f:
        for line in f.readlines():
            corpus.append(line)
            labels.append(0)

    return corpus, labels


# 过滤空文档
def remove_empty_docs(corpus, labels):
    filtered_corpus = []
    filtered_labels = []

    for doc, label in zip(corpus, labels):
        if doc.strip():
            filtered_corpus.append(doc)
            filtered_labels.append(label)

    return filtered_corpus, filtered_labels


# 计算并打印分类指标
def print_metrics(true_labels, predicted_labels):
    # Accuracy
    accuracy = metrics.accuracy_score(true_labels, predicted_labels)

    # Precision
    precision = metrics.precision_score(true_labels,
                                        predicted_labels,
                                        average='weighted')

    # Recall
    recall = metrics.recall_score(true_labels,
                                  predicted_labels,
                                  average='weighted')

    # F1
    f1 = metrics.f1_score(true_labels,
                          predicted_labels,
                          average='weighted')

    print("正确率: %.2f, 查准率: %.2f, 召回率: %.2f, F1: %.2f" % (accuracy, precision, recall, f1))

In [6]:
if __name__ == "__main__":
    global stopword_list

    # 读取停用词
    with open("stop_words.utf8", encoding="utf8") as f:
        stopword_list = f.readlines()

    corpus, labels = get_data()  # 加载数据
    corpus, labels = remove_empty_docs(corpus, labels)
    print("总的数据量:", len(labels))

    # 打印前N个样本
    for i in range(10):
        print("label:", labels[i], " 邮件内容:", corpus[i])

    # 对数据进行划分
    train_corpus, test_corpus, train_labels, test_labels = ms.train_test_split(corpus,
                                                                               labels,
                                                                               test_size=0.10,
                                                                               random_state=36)

    # 规范化处理
    norm_train_corpus = normalize_corpus(train_corpus)
    norm_test_corpus = normalize_corpus(test_corpus)

    # tfidf 特征
    ## 先计算tf-idf
    tfidf_vectorizer, tfidf_train_features = tfidf_extractor(norm_train_corpus)
    ## 再用刚刚训练的tf-idf模型计算测试集tf-idf
    tfidf_test_features = tfidf_vectorizer.transform(norm_test_corpus)
    # print(tfidf_test_features)
    # print(tfidf_test_features)

    # 基于tfidf的多项式朴素贝叶斯模型
    print("基于tfidf的贝叶斯模型")
    nb_model = MultinomialNB()  # 多分类朴素贝叶斯模型
    nb_model.fit(tfidf_train_features, train_labels)  # 训练
    mnb_pred = nb_model.predict(tfidf_test_features)  # 预测
    print_metrics(true_labels=test_labels, predicted_labels=mnb_pred)  # 打印测试集下的分类指标

    print("")

    # 基于tfidf的支持向量机模型
    print("基于tfidf的支持向量机模型")
    svm_model = SGDClassifier()  #分类器模块，没有指定会默认使用SVM
    svm_model.fit(tfidf_train_features, train_labels)  # 训练
    svm_pred = svm_model.predict(tfidf_test_features)  # 预测
    print_metrics(true_labels=test_labels, predicted_labels=svm_pred)  # 打印测试集下的分类指标

    print("")

    # 打印测试结果
    num = 0
    for text, label, pred_lbl in zip(test_corpus, test_labels, svm_pred):
        print('真实类别:', label_name_map[int(label)], ' 预测结果:', label_name_map[int(pred_lbl)])
        print('邮件内容【', text.replace("\n", ""), '】')
        print("")

        num += 1
        if num == 10:
            break

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/xw/r4jf50z15dv7m1bfg3f55mcw0000gn/T/jieba.cache


总的数据量: 10001
label: 1  邮件内容: 讲的是孔子后人的故事。一个老领导回到家乡，跟儿子感情不和，跟贪财的孙子孔为本和睦。 老领导的弟弟魏宗万是赶马车的。 有个洋妞大概是考察民俗的，在他们家过年。 孔为本总想出国，被爷爷教育了。 最后，一家人基本和解。 顺便问另一类电影，北京青年电影制片厂的。中越战背景。一军人被介绍了一个对象，去相亲。女方是军队医院的护士，犹豫不决，总是在回忆战场上负伤的男友，好像还没死。最后 男方表示理解，归队了。

label: 1  邮件内容: 不至于吧，离开这个破公司就没有课题可以做了？ 谢谢大家的关心，她昨天晚上睡的很好。MM她自己已经想好了。见机行事吧，拿到相关的能出来做论文的材料，就马上辞职。 唉！看看吧，说不定还要各为XDJM帮出出找工作的主意呢。MM学通信的，哈尔滨工程大学的研究生，不想在哈碌碌无为的做设计，因此才出来的。先谢谢了啊。！！！ 本人语文不好，没加标点。辛苦那些看不懂的XDJM么了。

label: 1  邮件内容: 生一个玩玩，不好玩了就送人 第一，你要知道，你们恋爱前，你爹妈对她是毫无意义的。没道理你爹妈就要求她生孩子，她就得听话。换句话说，你岳父母要未来孩子跟妈姓，你做的到吗？夫妻是平等的。如果你没办法答应岳父母，她干吗答应你爹妈呢？ 第二，有了孩子你养不养的起？不是说想生就生，图你爹妈一个高兴，如果没有房子，没有充足的财力，生孩子只会带给你们更多的困难，生小孩容易，养小孩难啊。

label: 1  邮件内容: 微软中国研发啥？本地化？ 新浪科技讯 8月24日晚10点，微软中国对外宣布说，在2006财年(2005年7月-2006年6月)，公司将在中国招聘约800名新员工。 其中，一半以上的新聘人员将为研发人员，其他将是销售、市场和服务人员。同时，有近300个职位将面向新毕业的大学本科生、硕士研究生、MBA和博士生。 在2005财年，微软在中国的业务取得了骄人成绩，成为微软全球增长速度最快的子公司之一。

label: 1  邮件内容: 要是他老怕跟你说话耽误时间 你可得赶紧纠正他这个观点 标  题: Re: 今天晚上的事情，有点郁闷 这个...其实以前有问题的时候都是当面解决，后来他说你有什么想不通的可以到板上去 问问别人，然后你就知道是谁不对了，所以这次我就来问，我觉得挺好，避免正面冲突， 

Loading model cost 1.009 seconds.
Prefix dict has been built successfully.


基于tfidf的贝叶斯模型
正确率: 1.00, 查准率: 1.00, 召回率: 1.00, F1: 1.00

基于tfidf的支持向量机模型
正确率: 1.00, 查准率: 1.00, 召回率: 1.00, F1: 1.00

真实类别: 正常邮件  预测结果: 正常邮件
邮件内容【 发信人: iliuzhongqi (我爱清华，更爱我的小猪), 信区: SEM.THU 标  题: 咨询一个相关分析的问题 我看到某权威期刊上的一篇论文， 做1952到2002年中国 “GDP――人口死亡率”之间的相关性验证 对数据没有作任何处理 用的是pearson相关系数 GDP和人口死亡率都不是正态分布的数据 能用pearson相关系数来验证吗？ 似乎应该用等级相关分析呀？ 】

真实类别: 垃圾邮件  预测结果: 垃圾邮件
邮件内容【 尊敬的客户您好: 中国东方航空公司上海售票处销售国内外特价机票,3-8折, 可随时来电来邮咨询,上海市内24小时免费送票 地址:上海市闸北区天目东路206号山水航空服务有限公司 电话:021-51270965 51270967 51270969 传真:51270969 QQ:76482909             MSN:majunxang8888@msn.com E-mail  majunxiang8888@126.com 】

真实类别: 正常邮件  预测结果: 正常邮件
邮件内容【 sigh，又来ile 当你谈恋爱的时候，你会觉得无比的甜蜜和幸福。因为能够到城里来念大学、和你相识的男孩子，都是从农村跳龙门走出来的优秀分子，他们坚定、上进、能吃苦、有一定的阅历，和他们在一起总是会对生活充满憧憬。 但这一切只能局限在纯粹的感情层面上，当你们要从恋爱进入“结婚”，走进现实生活的时候，就会发现原来感情在现实的力量面前远没有你所想象的那么强大。 第一眼见到他，你可能已经被他迷住了。 】

真实类别: 正常邮件  预测结果: 正常邮件
邮件内容【 其实怕的是一大群穷亲戚要你们小俩口资助以及农村父母的医疗问题,这又不是 风花雪月其乐融融的事,生病了你治还是不治,到那时候小孩要上学要供房子要 生活要支付大笔大笔的医疗费,压得你能喘得过气来吗?没这心理准备最好别想 的太美好 我是城市的姑娘，真搞不董为什么大家这么在乎农村的gg？ 有什么麻烦的？本来就该

In [10]:
print(tfidf_train_features.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
